# Code Chapter 01

## Typical imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy

In [ ]:
import arviz
import pymc3

## Thinking probabilistically

### Probability distributions

Investigating the normal distribution with Python

In [ ]:
µ = 0.
σ = 1.
X = scipy.stats.norm(µ, σ)
x = X.rvs(1000)

In [ ]:
scipy.stats.describe(x).mean

In [ ]:
(x < 1.96).sum() - (x < -1.97).sum()

The Gaussian (Normal) distribution

In [ ]:
mu_params = [-1, 0, 1]
sd_params = [0.5, 1, 1.5]
x = np.linspace(-7, 7, 100)

_, ax = plt.subplots(len(mu_params), len(sd_params),
                     sharex='all', sharey='all',
                     figsize=(9, 7), constrained_layout=True)
for i in range(len(mu_params)):
    for j in range(len(sd_params)):
        mu = mu_params[i]
        sd = sd_params[j]
        # Sample a normal distribution at all x
        y = scipy.stats.norm(mu, sd).pdf(x)
        ax[i, j].plot(x, y)
        ax[i, j].plot([], label=f'μ={mu:3.2f}\nσ={sd:3.2f}', alpha=0)
        ax[i, j].legend(loc=1)
ax[2, 1].set_xlabel('x')
ax[1, 0].set_ylabel('p(x)', rotation=0, labelpad=20)
ax[1, 0].set_yticks([])  # no `xticks` because values not necessary for understanding